# Specimen Search Interface

A brief overview of how to make simple queries against the specimen database(s) for
the Herbarium's Michigan Flora website.

***
Note:
Due to the organization of the project itself, the notebooks being stored one level
deeper than the rest of the project, we need to add the parent folder to the path so
our custom packages can be located.  This is not needed in the main Flask app.

In [1]:
import sys
sys.path.append("../")

***
### Imports

The principle interface to the Michigan Flora database is found in our 'db.miflora'.
This is the main object through which we'll perform our searches.

In [2]:
from db.miflora import MIFloraDB

To directly access the table descriptions and their associated columns, importing the
table class objects is needed. The table class objects are the ORM representation of
the tables of the database.  

In [3]:
import db.miflora as mf

Specimen records are returned in a standard format regardless of the database backend.  Currently, this record is simply
a namedtuple with a basic set of fields, though as time goes on more functionlity will likely be added, such as helper
functions to marshal the data into other formats, e.g. JSON.

In [4]:
from db.core import SpecimenSearchRecord

Here we import any installed packages, i.e. not our own packages, that we may need to properly run this notebook.
If nothing else, this section can be a placeholder for future notebook expansion.

In [5]:
import sqlparse

***
### Connecting to the database.

First we need to construct the database object and pass it our credentials file containing the username and password. 
This file is stored in **pickle** binary format.

In [6]:
credentials_file = "../mif_creds.p"
miflora_db = MIFloraDB(credentials_file=credentials_file)

We haven't actually connected to the database, so we need to kickstart the engine.

In [7]:
miflora_db.kickstart()

***
### Anatomy of a Specimen Search


The specimen search is consolidated in single class method of our database object: **search_specimens**.  Internally,
this method generates the necessary SQL query or queries via SQLAlchemy.  Currently, the logic of the query is more or
less a reproduction of the raw SQL query that the old website runs.  This will likely change in near future as the new
Specify database becomes the standard.

All searches are composed of the following optional parameters.  All string parameters default to an empty string,
which effectively acts like a * wildcard.  In other words, by not specifying a parameter, you will be running an unbound
search with no constraints.  This will demonstrated more clearly in the search examples below.

Notice there are 2 exceptions to the string rule, **n_results** and **offset**.  These two parameters limit the number
of records returned and provide an offset to skip the first N records.  Combining these two parameters allow you to
paginate the search results.

#### Search Parameters
* common_name (str)
* scientific_name (str)
* genus (str)
* family (str)
* collector_name (str)
* collector_number (str)
* collection_year (str)
* n_results (integer), default: 25
* offset (integer), default: 0

Specimen searches return a list of matches where each element of the list is an object of type SpecimenSearchRecord
(namedtuple).  Each record should be a unique result of the query with no duplicates.

#### SpecimenSearchRecord Fields
* 'identity'
* 'catalog_number',
* 'family'
* 'genus'
* 'collectors'
* 'place_name'
* 'species_epithet'
* 'infra_rank'
* 'infra_name'
* 'identification_qualifier'
* 'plant_id'
* 'status'
* 'county_district'
* 'collectors_number'
* 'collection_date'
* 'locality'

You might be looking at that list and wondering what happend to the common name.  As it turns out, the common name can
vary and multiple common names can be attached to the exact same species and variant of plant.  The Herbarium folks
decided they did not want to see this duplication in the results, so for all intents and purposes,
multiple records that are the same in all ways except the common name are the same record.

##### Developer's Note: Currently, I don't actually know what all these fields are used for as many of them were connected to the legacy database setup, but may change in the near future as everything is migrated to the Specify export database.  I am working with Bev Walters (head at Herbarium) to determine which fields are useful, and potentially new fields to add in order to make the search queries more useful.

***
### Search Examples
Here we'll demonstrate the basics of running a specimen search via the database object **MIFLORA_DB** and inspecting
the results.

First, we'll setup some handy helper functions to make looking at the specimen search results a little easier.

In [8]:
def print_record_count(specimen_records):
    print("Number of records: {0}".format(len(specimen_records)))
    
def print_records(specimen_records):
    i = 1
    
    for r in specimen_records:
        print("[{0}] Family: {1}, Genus: {2}, Collector(s): {3}".format(i, r.family, r.genus, r.collectors))
        i += 1
        
def pretty_sql_print(query):
    print(sqlparse.format(str(query), reindent=True, keyword_case='upper'))

At its most basic, a search comprises of a set of constrainsts.  Most of the constraints operate like a
case-insensitive wildcard match where we're just looking for the presence of the string anywhere in a given database
column.  For example, searching for a common name of "MAPLE" or "mAplE" will match a column containing
"Super Cool Maple Tree".  The exception to this is the **collector_number** field which does an exact string match.

Let's start out with a pretty typical search

In [9]:
specimen_records = miflora_db.search_specimens(common_name='Maple', collection_year='1952', collector_name='Voss')
print_record_count(specimen_records)

Number of records: 5


Now let's take a peek at the results.

In [10]:
specimen_records

[SpecimenSearchRecord(identity=129622, catalog_number='1246675', family='Adoxaceae', genus='Viburnum', collectors='Edward G. Voss', place_name='Huron County', species_epithet='acerifolium', infra_rank=None, infra_name=None, identification_qualifier=None, plant_id=13, status=None, county_district='Huron', collectors_number='1122', collection_date='05/11/1952', locality='"Wilderness Arboretum," ca. 14 miles NW of Bad Axe  in sec. 7, Hume Township.)'),
 SpecimenSearchRecord(identity=293250, catalog_number='1246843', family='Adoxaceae', genus='Viburnum', collectors='Edward G. Voss', place_name='Huron County', species_epithet='acerifolium', infra_rank=None, infra_name=None, identification_qualifier=None, plant_id=13, status=None, county_district='Huron', collectors_number='1468', collection_date='09/27/1952', locality='"Wilderness Arboretum," ca. 14 miles NW of Bad Axe  in sec. 7, Hume Township.)'),
 SpecimenSearchRecord(identity=359264, catalog_number=None, family='Sapindaceae', genus='Ace

Woah!  Too much info all at once!  If you're using the PyCharm debugger, then you can easily browse the data structures,
but in a notebook we need to do something different.

Looking at individual records as dictionaries is much more friendly.

In [11]:
specimen_records[0]._asdict()

OrderedDict([('identity', 129622),
             ('catalog_number', '1246675'),
             ('family', 'Adoxaceae'),
             ('genus', 'Viburnum'),
             ('collectors', 'Edward G. Voss'),
             ('place_name', 'Huron County'),
             ('species_epithet', 'acerifolium'),
             ('infra_rank', None),
             ('infra_name', None),
             ('identification_qualifier', None),
             ('plant_id', 13),
             ('status', None),
             ('county_district', 'Huron'),
             ('collectors_number', '1122'),
             ('collection_date', '05/11/1952'),
             ('locality',
              '"Wilderness Arboretum," ca. 14 miles NW of Bad Axe  in sec. 7, Hume Township.)')])

This is OK, but we only see one record at a time and using this format with lots of records might be unwieldy. Instead,
we can print out a more concise list of all the records using our handy helper function defined above.

#### Developer's Note: Currently, I'm working on a pretty print function to iterate over a set of records and print them out as an ASCII table, but this is not complete.  The 'tabulate' package looks pretty promising.

In [12]:
print_records(specimen_records)

[1] Family: Adoxaceae, Genus: Viburnum, Collector(s): Edward G. Voss
[2] Family: Adoxaceae, Genus: Viburnum, Collector(s): Edward G. Voss
[3] Family: Sapindaceae, Genus: Acer, Collector(s): Edward G. Voss
[4] Family: Sapindaceae, Genus: Acer, Collector(s): Edward G. Voss
[5] Family: Amaranthaceae, Genus: Chenopodium, Collector(s): Edward G. Voss


#### Limiting the Number of Results

By default, the specimen search only returns a maximum of 25 results unless otherwise specified.  The idea being that
the database is huge, and a lazy query might unintentially return tens of thousands of results, thus needlessly taxing
the server(s).

We can change the maximum number of results by modifying the **n_results** parameter to anything in the range [0,N]
where N can be any positive integer value.
__Note: 0 is a special number in that it tells the search function to return **all** records.__

Here are a few examples varying the number of results using the same search.

In [15]:
specimen_records = miflora_db.search_specimens(n_results=25, collection_year='1952')
print_record_count(specimen_records)

Number of records: 25


In [ ]:
specimen_records = miflora_db.search_specimens(n_results=200, collection_year='1952')
print_record_count(specimen_records)

In [ ]:
specimen_records = miflora_db.search_specimens(n_results=0, collection_year='1952')
print_record_count(specimen_records)

#### Paginating the Results
That last search probably has more records than we want at once.  In addition, we do not want to needlessly tax the
server when we most likely want to see chunks of records.  This is important when displaying results on a webpage.

Pagination the act of chunking up the results, and is easy to do by using both the **n_results** and **offset**
parameters.

Let's paginate that search by retrieving the first 10 records in 2 separate chunks (for display purposes).

In [ ]:
chunk1 = miflora_db.search_specimens(collection_year='1952', n_results=10, offset=0)
chunk2 = miflora_db.search_specimens(collection_year='1952', n_results=10, offset=10)

In [ ]:
print_records(chunk1)

In [ ]:
print_records(chunk2)

#### Counting Records
Sometimes you want to know how many results a particular query will give without making the search itself.

In [ ]:
n_records = miflora_db.specimen_search_count(collection_year='1952', n_results=0)
print('Number of records: {0}'.format(n_records))

#### Viewing the Raw SQL

All searching and counting functions rely on a underlying method that generates the SQL that is executed.  This lets you
either inspect the SQL before running a query or allowing you to manipulate the SQL prior to running a search.

In [ ]:
query = miflora_db.make_specimen_search(collection_year='1952', n_results=2)

The resultant query value is actually a [SQLAlchemy Query](https://docs.sqlalchemy.org/en/13/orm/query.html) object, so
you can run any of the functions provided by the API in the event you want total control of the object.

Printing the query directly will give the the raw SQL, but it's a bit ugly.

In [ ]:
print(query)

Buy using the [sqlparse](https://pypi.org/project/sqlparse/) package we can make it much easier to read.  See the method
definition section above for how the package is being used here.

In [ ]:
pretty_sql_print(query)